[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/schlichtanders/fall-in-love-with-julia/main?filepath=02%20deeplearning%20using%20flux%20-%2002%20trebuchet.ipynb)

# Controlling a Trebuchet

![trebuchet](https://fluxml.ai/assets/2019-03-05-dp-vs-rl/trebuchet-basic.gif)

There is Trebuchet, which throws a mass to a target. The mass is to be
released at an angle, and at certain velocity so that it lands on the target.
The velocity of release is determined by the counterweight of the Trebuchet.
Given conditions of environment we are required to predict the angle of
release and counterweight.

* **Input:**  Wind speed,   Target distance
* **Output:** ReleaseAngle, Weight

![overview](https://fluxml.ai/assets/2019-03-05-dp-vs-rl/trebuchet-flow.png)

In [ ]:
using Flux
using Random
using Plots
plotlyjs()
import Trebuchet
import Zygote

# linear interpolation helper
lerp(x, lo, hi) = x*(hi-lo)+lo

# there is currently an issue with Zygote.ignore, luckily a self-written version still works
# for an update consult https://github.com/FluxML/Zygote.jl/issues/677 
gradient_ignore(f) = f()
Zygote.@adjoint gradient_ignore(f) = gradient_ignore(f), _ -> nothing

In [ ]:
function visualize_trebuchet(;target=100, wind_speed=1.0, release_angle=45, weight=98.09)  # default values from TrebuchetState
    # state is going to be mutated by simulate, hence we capsulate it into our own method
    release_angle = Trebuchet.deg2rad(release_angle)
    state = Trebuchet.TrebuchetState(wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    Trebuchet.simulate(state)  # should be named `simulate!(t)`
    Trebuchet.visualise(state, target)
end 

function shoot_trebuchet(;wind_speed=1.0, release_angle=45, weight=98.09)
    release_angle = Trebuchet.deg2rad(release_angle)
    state = Trebuchet.TrebuchetState(;wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    weight > 0 || return 0.0
    Trebuchet.simulate(state)
    Trebuchet.endDist(state)
end

In [ ]:
visualize_trebuchet(target=50)

In [ ]:
shoot_trebuchet()

## Create Model

In [ ]:
Random.seed!(0)
model = Chain(Dense(2, 16, σ),
              Dense(16, 64, σ),
              Dense(64, 16, σ),
              Dense(16, 2)) |> f32
θ = params(model)

## Create Loss

As Trebuchet.shoot uses `try`/`catch` within, which is not yet supported by default backwards differentiation, we need to mark our shoot function with ``Zygote.forwarddiff`` marker.

In [ ]:
function aim(wind, target)
  angle, weight = model([wind, target])
  angle = σ(angle)*90
  weight = weight + 200
  (release_angle=angle, weight=weight)
end

function visualize_model(;wind_speed=1.0, target=100)
    release_angle, weight = aim(wind_speed, target)
    visualize_trebuchet(target=target, wind_speed=wind_speed, release_angle=release_angle, weight=weight)
end

function shoot_model(;wind_speed=1.0, target=100)
    release_angle, weight = aim(wind_speed, target)
    # shoot_trebuchet uses array mutation internally, which is not yet supported by Zygote ReverseDiff
    # however forwarddiff works with everything, including array mutation and try/catch,
    # hence we mark this respectively
    Zygote.forwarddiff([wind_speed, release_angle, weight]) do (wind_speed, release_angle, weight)
        shoot_trebuchet(wind_speed=wind_speed, release_angle=release_angle, weight=weight)
    end
end

In [ ]:
visualize_model(wind_speed=1.0, target=50)

In [ ]:
shoot_model(wind_speed=1.0, target=50)

## Train

In [ ]:
target_min, target_max = 20, 100	# Maximum target distance
wind_speed_mean = 5 # Maximum wind speed

random_target() = (
    wind_speed = randn() * wind_speed_mean,
    target = lerp(rand(), target_min, target_max)
)

In [ ]:
losses = Float64[]
iterations = Int[]
i = 0

In [ ]:
optimizer = ADAM()
try
    while true
        i += 1
        wind_speed, target = random_target()
        ∇θ = gradient(θ) do
            hit = shoot_model(wind_speed=wind_speed, target=target)
            loss = (hit - target)^2
            gradient_ignore() do
                if i % 100 == 0
                    push!(losses, loss)
                    push!(iterations, i)
                    plot(iterations, losses, show = :inline, yscale = :log10,
                        label = "square-loss", xlabel = "#iteration", ylabel="loss (log10 scale)")
                end
            end
            loss
        end
        Flux.update!(optimizer, θ, ∇θ)
    end
    
catch e
    if e isa InterruptException
        visualize_model(;random_target()...)
    end
end 

In [ ]:
visualize_model(;random_target()...)

# Thank you

For more details and further examples see the original blogpost  https://fluxml.ai/2019/03/05/dp-vs-rl.html.

Or ask me directly at stephan.sahm@gmx.de.